<a href="https://colab.research.google.com/github/ManuelVR461/DuocUCProyectoFinal/blob/master/mercadopublico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis Base de datos

1. [X] Paso Identificar la base de datos y las columnas a utilizar (2)
2. [ ] Obtener el rango de fechas a utilizar y unificarlos en una sola data (1)
3. [ ] Proceder con la Limpieza y tranformacion los datos. (1)
4. [ ] Determinar la pregunta en question (2)
5. [ ] Identificar el modelo a utilizar. (2)
6. [ ] Generar los datos de prediccion de tres meses.(1)
7. [ ] Generar la estrella. (3)
8. [ ] Generar El cubo. (3)
9. [ ] Generar El tablero de PowerBi. (4)
10.[ ] Preparar el history telling.


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import unicodedata

from io import BytesIO
import zipfile
import requests

%matplotlib inline

In [ ]:
# Lista de nombres de archivos ZIP a descargar
anio = "2023"
meses = 9
archivos_a_descargar = []

for mes in range(1, meses + 1):
  nombre_archivo = f"{anio}-{mes}.zip"
  archivos_a_descargar.append(nombre_archivo)

print(archivos_a_descargar)

['2023-1.zip', '2023-2.zip', '2023-3.zip', '2023-4.zip', '2023-5.zip', '2023-6.zip', '2023-7.zip', '2023-8.zip', '2023-9.zip']


In [ ]:
directorio_archivos = '/content/data_projecto_final/'+anio
directorio_tablas_dinamicas = '/content/data_projecto_final/tablas_dinamicas/'+anio
directorio_tablas_hecho = '/content/data_projecto_final/tablas_hecho/'+anio

if not os.path.exists(directorio_archivos):
    os.makedirs(directorio_archivos)

if not os.path.exists(directorio_tablas_dinamicas):
    os.makedirs(directorio_tablas_dinamicas)

if not os.path.exists(directorio_tablas_hecho):
    os.makedirs(directorio_tablas_hecho)

In [ ]:
# Bucle para descargar y extraer los archivos
for archivo in archivos_a_descargar:
    url = f"https://transparenciachc.blob.core.windows.net/lic-da/{archivo}"
    response = requests.get(url)
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        z.extractall(path=directorio_archivos)

In [ ]:
# Inicializar una lista para almacenar los DataFrames individuales
dataframes = []

# Obtener la lista de archivos CSV en el directorio
archivos_csv = [archivo for archivo in os.listdir(directorio_archivos) if archivo.endswith(".csv")]

# Iterar a través de los archivos y cargarlos en DataFrames individuales
for archivo_csv in archivos_csv:
    ruta_archivo = os.path.join(directorio_archivos, archivo_csv)
    df_individual = pd.read_csv(ruta_archivo, encoding="ISO-8859-1", delimiter=";")
    #df = pd.read_csv("/content/lic_2023-9.csv", encoding="ISO-8859-1", error_bad_lines=False, delimiter=";")
    dataframes.append(df_individual)

# Combinar todos los DataFrames individuales en uno solo
df = pd.concat(dataframes, ignore_index=True)
df.shape

<ipython-input-6-2a19e088322c>:10: DtypeWarning: Columns (63,87,97,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_individual = pd.read_csv(ruta_archivo, encoding="ISO-8859-1", delimiter=";")
<ipython-input-6-2a19e088322c>:10: DtypeWarning: Columns (27,52,63,64,87,97,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_individual = pd.read_csv(ruta_archivo, encoding="ISO-8859-1", delimiter=";")
<ipython-input-6-2a19e088322c>:10: DtypeWarning: Columns (27,52,63,64,87,94,97,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_individual = pd.read_csv(ruta_archivo, encoding="ISO-8859-1", delimiter=";")
<ipython-input-6-2a19e088322c>:10: DtypeWarning: Columns (27,52,54,63,64,87,94,97,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  df_individual = pd.read_csv(ruta_archivo, encoding="ISO-8859-1", delimiter=";")
<ipython-input-6-2a19e088322c>:10: DtypeWarning:

(1479006, 105)

In [ ]:
# Verificamos los nombres de columnas
df.columns

Index(['Codigo', 'Link', 'CodigoExterno', 'Nombre', 'Descripcion',
       'Tipo de Adquisicion', 'CodigoEstado', 'Estado', 'CodigoOrganismo',
       'NombreOrganismo',
       ...
       'Nombre de la Oferta', 'Estado Oferta', 'Cantidad Ofertada',
       'Moneda de la Oferta', 'MontoUnitarioOferta', 'Valor Total Ofertado',
       'CantidadAdjudicada', 'MontoLineaAdjudica', 'FechaEnvioOferta',
       'Oferta seleccionada'],
      dtype='object', length=105)

In [ ]:
# Procedemos a limpiar los espacios en blanco que pudieran estar en los nombres de las columnas
df.columns = df.columns.str.strip()

In [ ]:
# Convertir campos vacíos o sin registros en None en cada columna
df = df.applymap(lambda x: None if pd.isna(x) or x == '' else x)

In [ ]:
# Validar la cantidad de duplicados en el DataFrame
cantidad_duplicados = df.duplicated().sum()
# Imprimir la cantidad de duplicados
print("Cantidad de duplicados en el DataFrame:", cantidad_duplicados)

Cantidad de duplicados en el DataFrame: 383


In [ ]:
# Eliminar las filas duplicadas del DataFrame
df = df.drop_duplicates()
df.shape

(1478623, 105)

In [ ]:
df.head(5)

,Codigo,Link,CodigoExterno,Nombre,Descripcion,Tipo de Adquisicion,CodigoEstado,Estado,CodigoOrganismo,NombreOrganismo,...,Nombre de la Oferta,Estado Oferta,Cantidad Ofertada,Moneda de la Oferta,MontoUnitarioOferta,Valor Total Ofertado,CantidadAdjudicada,MontoLineaAdjudica,FechaEnvioOferta,Oferta seleccionada
0,8992351,http://www.mercadopublico.cl/fichaLicitacion.h...,690263-11-LR22,CONVENIO SUMINISTRO DE REACTIVOS E INSUMOS DE ...,Proveer el suministro de reactivos e insumos p...,Licitación Pública Mayor a 5000 (LR),6,Cerrada,6976,SERVICIO DE SALUD ARAUCO,...,CONVENIO SUMINISTRO DE REACTIVOS E INSUMOS DE ...,Rechazada,1,Peso Chileno,3998400000,3998400000,0,0,2023-07-11,No Seleccionada
1,8992351,http://www.mercadopublico.cl/fichaLicitacion.h...,690263-11-LR22,CONVENIO SUMINISTRO DE REACTIVOS E INSUMOS DE ...,Proveer el suministro de reactivos e insumos p...,Licitación Pública Mayor a 5000 (LR),6,Cerrada,6976,SERVICIO DE SALUD ARAUCO,...,Cotización N° 312326,Aceptada,1,Peso Chileno,1,1,0,0,2023-08-08,No Seleccionada
2,9050946,http://www.mercadopublico.cl/fichaLicitacion.h...,886954-286-LR22,Convenio de suministro Prótesis Para la Unidad...,La necesidad de abastecer del insumo para la a...,Licitación Pública Mayor a 5000 (LR),6,Cerrada,7324,HOSPITAL GUILLERMO GRANT BENAVENTE DE CO,...,Prótesis Para la Unidad de Medicina Fisica y R...,Aceptada,1,Peso Chileno,25000,25000,0,0,2023-08-10,No Seleccionada
3,9050946,http://www.mercadopublico.cl/fichaLicitacion.h...,886954-286-LR22,Convenio de suministro Prótesis Para la Unidad...,La necesidad de abastecer del insumo para la a...,Licitación Pública Mayor a 5000 (LR),6,Cerrada,7324,HOSPITAL GUILLERMO GRANT BENAVENTE DE CO,...,CONVENIO SUMINISTRO DE PROTESIS PARA UNIDAD ME...,Aceptada,1,Peso Chileno,25000,25000,0,0,2023-08-10,No Seleccionada
4,9050946,http://www.mercadopublico.cl/fichaLicitacion.h...,886954-286-LR22,Convenio de suministro Prótesis Para la Unidad...,La necesidad de abastecer del insumo para la a...,Licitación Pública Mayor a 5000 (LR),6,Cerrada,7324,HOSPITAL GUILLERMO GRANT BENAVENTE DE CO,...,Propuesta para Convenio de suministro Prótesis...,Aceptada,1,Peso Chileno,35000,35000,0,0,2023-08-09,No Seleccionada


## **Posibles campos para las tablas dimensionales**

1. dim_desc_licitaciones (Codigo, Nombre, Descripcion)
2. dim_tipo_de_adquisicion (id_tipo, Tipo de Adquisicion,)
3. dim_estados (CodigoEstado, Estado)
4. dim_sector_organismo (id_sector, sector)
5. dim_organismo (CodigoOrganismo, NombreOrganismo, RutUnidad, id_sector)
6. dim_unidad (CodigoUnidad, NombreUnidad)
7. dim_comuna (id_comuna, ComunaUnidad)
8. dim_region (id_region, RegionUnidad)
9. dim_rubros1 (id_rubro1, Rubro1)
10. dim_rubros2 (id_rubro2, Rubro2)
11. dim_rubros3 (id_rubro3, Rubro3)

## Tabla de Hecho
* Fact_licitaciones:
    Codigo, id_tipo, CodigoEstado, CodigoOrganismo, id_sector, CodigoUnidad, id_comuna,id_region, Tipo, Etapas, EstadoPublicidadOfertas, Contrato, CantidadReclamos,FechaCreacion, FechaCierre, FechaInicio, FechaFinal, FechaPubRespuestas, FechaAdjudicacion, etc, id_rubro1,id_rubro2,id_rubro3,)

In [ ]:
# Metodo para Normalizar los caracteres especiales que pudieran existir en cada tabla dimencional creada
def normalize_text(text):
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode()

#### 1. Tabla **dim_desc_licitaciones** *(CodigoExterno, Nombre, Descripcion*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['CodigoExterno'].isnull().sum()
cant_nulos

0

In [ ]:
# Crear un nuevo DataFrame con valores únicos
df_dim_desc_licitaciones = df[['CodigoExterno', 'Nombre', 'Descripcion']].drop_duplicates()
df_dim_desc_licitaciones.head(5)

,CodigoExterno,Nombre,Descripcion
0,690263-11-LR22,CONVENIO SUMINISTRO DE REACTIVOS E INSUMOS DE ...,Proveer el suministro de reactivos e insumos p...
2,886954-286-LR22,Convenio de suministro Prótesis Para la Unidad...,La necesidad de abastecer del insumo para la a...
150,2767-97-LE22,ADQUISICIÓN DE 5 CONTAINER PARA SALUD LA CISTERNA,La Municipalidad de La Cisterna llama a Propue...
154,2111-2-LE23,SUMINISTRO DE BOLSAS FRASCO ETIQUETA PARA BIOPSIA,ASEGURAR EL ABASTECIMIENTO DE INSUMOS PARA CUB...
163,1704-2-LQ23,SERVICIO DE APOYO ADMINISTRATIVO PARA EL HOSPITAL,APOYAR A LAS UNIDADES INVOLUCRADAS A ABORDAR E...


In [ ]:
df_dim_desc_licitaciones.shape

(82550, 3)

In [ ]:
#Eliminamos las columnas del dataframe principal y se deja solo la columna CodigoExterno
#de esta forma vamos avanzando en la limpieza y reduccion de los datos de la tabla de hecho
df.drop(["Link","Nombre","Descripcion"],axis='columns',inplace=True)
df.shape

(1478623, 102)

In [ ]:
df_dim_desc_licitaciones.dtypes

CodigoExterno    object
Nombre           object
Descripcion      object
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_desc_licitaciones.select_dtypes(include=['object']):
    print(columna)
    df_dim_desc_licitaciones[columna] = df_dim_desc_licitaciones[columna].apply(normalize_text)

CodigoExterno
Nombre
Descripcion


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_desc_licitaciones.to_csv(directorio_tablas_dinamicas + "/dim_desc_licitaciones.csv", index=False)

#### 2. Tabla **dim_tipo_de_adquisicion** *(id_tipo, Tipo de Adquisicion)*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['Tipo de Adquisicion'].isnull().sum()
cant_nulos

0

In [ ]:
# Obtener valores únicos y eliminar duplicados
unicos_adquisicion = df['Tipo de Adquisicion'].unique()

# Crear un nuevo DataFrame con dos columnas "id" y "valor_unico"
df_dim_tipo_adquisicion = pd.DataFrame({'id_tipo': range(1, len(unicos_adquisicion) + 1),
                         'TipoAdquisicion': unicos_adquisicion})

df_dim_tipo_adquisicion.head(5)

,id_tipo,TipoAdquisicion
0,1,Licitación Pública Mayor a 5000 (LR)
1,2,Licitación Pública Entre 100 y 1000 UTM (LE)
2,3,Licitación Pública entre a 2000 y 5000 UTM (LQ)
3,4,Licitación Pública Mayor 1000 UTM (LP)
4,5,Licitación Pública Menor a 100 UTM (L1)


In [ ]:
df_dim_tipo_adquisicion.shape

(13, 2)

In [ ]:
#reemplazamos los valores de la columna Tipo de Adquisicion de la tabla principal por los IDs de los valores unicos
df['Tipo de Adquisicion'] = df['Tipo de Adquisicion'].replace(df_dim_tipo_adquisicion.set_index('TipoAdquisicion')['id_tipo'])

#Se renombra el titulo de la columna para mejor comprension del cambio
df.rename(columns={'Tipo de Adquisicion': 'id_tipo'}, inplace=True)

#Seteamos a entero el campo
df['id_tipo'] = df['id_tipo'].astype(int)

df.shape


(1478623, 102)

In [ ]:
df_dim_tipo_adquisicion.dtypes

id_tipo             int64
TipoAdquisicion    object
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_tipo_adquisicion.select_dtypes(include=['object']):
    print(columna)
    df_dim_tipo_adquisicion[columna] = df_dim_tipo_adquisicion[columna].apply(normalize_text)

TipoAdquisicion


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_tipo_adquisicion.to_csv(directorio_tablas_dinamicas + "/dim_tipo_adquisicion.csv", index=False)

#### 3. Tabla **dim_estados** *(CodigoEstado, Estado)*

In [ ]:
#aun no tenemos datos

#### 4. Tabla **dim_sector_organismo** *(id_sector, sector)*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['sector'].isnull().sum()
cant_nulos

293

In [ ]:
df.shape

(1478623, 102)

In [ ]:
# Eliminar registros donde sector tenga valores vacíos
df = df.dropna(subset=['sector'])

In [ ]:
df.shape

(1478330, 102)

In [ ]:
# Obtener valores únicos y eliminar duplicados
unicos_sector = df['sector'].unique()
# Crear un nuevo DataFrame con dos columnas "id_sector" y "sector"
df_dim_sector_organismo = pd.DataFrame({'id_sector': range(1, len(unicos_sector) + 1),
                         'sector': unicos_sector})

df_dim_sector_organismo.head(5)

,id_sector,sector
0,1,Salud
1,2,Municipalidades
2,3,Obras Públicas
3,4,"Gob. Central, Universidades"
4,5,FFAA


In [ ]:
df_dim_sector_organismo.shape

(7, 2)

In [ ]:
#reemplazamos los valores de la columna sector de la tabla principal por los IDs de los valores unicos
df['sector'] = df['sector'].replace(df_dim_sector_organismo.set_index('sector')['id_sector'])
#Se renombra el titulo de la columna para mejor comprension del cambio
df.rename(columns={'sector': 'id_sector'}, inplace=True)
#Seteamos a entero el campo
df['id_sector'] = df['id_sector'].astype(int)
df.shape

(1478330, 102)

In [ ]:
df_dim_sector_organismo.dtypes

id_sector     int64
sector       object
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_sector_organismo.select_dtypes(include=['object']):
    print(columna)
    df_dim_sector_organismo[columna] = df_dim_sector_organismo[columna].apply(normalize_text)

sector


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_sector_organismo.to_csv(directorio_tablas_dinamicas + "/dim_sector_organismo.csv", index=False)

#### 5. Tabla **dim_organismo** *(CodigoOrganismo, NombreOrganismo, RutUnidad, id_sector)*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['CodigoOrganismo'].isnull().sum()
cant_nulos

0

In [ ]:
df['CodigoOrganismo'] = df['CodigoOrganismo'].astype(int)
# Crear un nuevo DataFrame con valores únicos
df_dim_organismos = df[['CodigoOrganismo', 'NombreOrganismo', 'RutUnidad','id_sector']].drop_duplicates()
df_dim_organismos.head(5)

,CodigoOrganismo,NombreOrganismo,RutUnidad,id_sector
0,6976,SERVICIO DE SALUD ARAUCO,61.954.500-1,1
2,7324,HOSPITAL GUILLERMO GRANT BENAVENTE DE CO,61.602.189-3,1
150,100488,I MUNICIPALIDAD DE LA CISTERNA,69.072.000-0,2
154,7460,HOSPITAL DE URGENCIA ASISTENCIA PUBLICA DR ALE...,61.608.602-2,1
163,7345,SERVICIO DE SALUD VINA DEL MAR QUILLOTA,61.606.608-0,1


In [ ]:
df_dim_organismos.shape

(1620, 4)

In [ ]:
#Eliminamos las columnas del dataframe principal y se deja solo la columna CodigoExterno
#de esta forma vamos avanzando en la limpieza y reduccion de los datos de la tabla de hecho
df.drop(['NombreOrganismo', 'RutUnidad','id_sector','DireccionUnidad'],axis='columns',inplace=True)
df.shape

(1478330, 98)

In [ ]:
df_dim_organismos.dtypes

CodigoOrganismo     int64
NombreOrganismo    object
RutUnidad          object
id_sector           int64
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_organismos.select_dtypes(include=['object']):
    print(columna)
    df_dim_organismos[columna] = df_dim_organismos[columna].apply(normalize_text)

NombreOrganismo
RutUnidad


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_organismos.to_csv(directorio_tablas_dinamicas + "/dim_organismos.csv", index=False)

#### 6. Tabla **dim_unidad** *(CodigoUnidad, NombreUnidad)*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['CodigoUnidad'].isnull().sum()
cant_nulos

0

In [ ]:
df['CodigoUnidad'] = df['CodigoUnidad'].astype(int)
# Crear un nuevo DataFrame con valores únicos
df_dim_unidad = df[['CodigoUnidad','NombreUnidad']].drop_duplicates()
df_dim_unidad.head(5)

,CodigoUnidad,NombreUnidad
0,690263,ABASTECIMIENTO FÁRMACOS E INSUMOS CLÍNICOS
2,886954,Abastecimiento Insumos
150,3756,SECPLAC
154,3106,Hospital de Urgencia Asistencia Pública
163,2700,Hospital San Martín de Quillota


In [ ]:
df_dim_unidad.shape

(3636, 2)

In [ ]:
#Eliminamos las columnas del dataframe principal y se deja solo la columna CodigoExterno
#de esta forma vamos avanzando en la limpieza y reduccion de los datos de la tabla de hecho
df.drop(['NombreUnidad'],axis='columns',inplace=True)
df.shape

(1478330, 97)

In [ ]:
df_dim_unidad.dtypes

CodigoUnidad     int64
NombreUnidad    object
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_unidad.select_dtypes(include=['object']):
    print(columna)
    df_dim_unidad[columna] = df_dim_unidad[columna].apply(normalize_text)

NombreUnidad


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_unidad.to_csv(directorio_tablas_dinamicas + "/dim_unidad.csv", index=False)

#### 7. Tabla **dim_comuna** *(id_comuna,ComunaUnidad)*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['ComunaUnidad'].isnull().sum()
cant_nulos

89095

In [ ]:
df.shape

(1478330, 97)

In [ ]:
# Eliminar registros donde sector tenga valores vacíos
df = df.dropna(subset=['ComunaUnidad'])

In [ ]:
df.shape

(1389235, 97)

In [ ]:
# Obtener valores únicos y eliminar duplicados
unicos_comuna = df['ComunaUnidad'].unique()
# Crear un nuevo DataFrame con dos columnas "id_comuna" y "comunaunidad"
df_dim_comuna = pd.DataFrame({'id_comuna': range(1, len(unicos_comuna) + 1),
                         'ComunaUnidad': unicos_comuna})

df_dim_comuna.head(5)

,id_comuna,ComunaUnidad
0,1,Lebu
1,2,Concepción
2,3,La Cisterna
3,4,Santiago
4,5,Illapel


In [ ]:
df_dim_comuna.shape

(346, 2)

In [ ]:
#reemplazamos los valores de la columna sector de la tabla principal por los IDs de los valores unicos
df['ComunaUnidad'] = df['ComunaUnidad'].replace(df_dim_comuna.set_index('ComunaUnidad')['id_comuna'])
#Se renombra el titulo de la columna para mejor comprension del cambio
df.rename(columns={'ComunaUnidad': 'id_comuna'}, inplace=True)
#Seteamos a entero el campo
df['id_comuna'] = df['id_comuna'].astype(int)
df.shape

(1389235, 97)

In [ ]:
df_dim_comuna.dtypes

id_comuna        int64
ComunaUnidad    object
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_comuna.select_dtypes(include=['object']):
    print(columna)
    df_dim_comuna[columna] = df_dim_comuna[columna].apply(normalize_text)

ComunaUnidad


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_comuna.to_csv(directorio_tablas_dinamicas + "/dim_comuna.csv", index=False)

#### 8. Tabla **dim_region** *(id_region,RegionUnidad)*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['RegionUnidad'].isnull().sum()
cant_nulos

0

In [ ]:
# Obtener valores únicos y eliminar duplicados
unicos_region = df['RegionUnidad'].unique()
# Crear un nuevo DataFrame con dos columnas "id_comuna" y "comunaunidad"
df_dim_region = pd.DataFrame({'id_region': range(1, len(unicos_region) + 1),
                         'RegionUnidad': unicos_region})

df_dim_region.head(5)

,id_region,RegionUnidad
0,1,Región del Biobío
1,2,Región Metropolitana de Santiago
2,3,Región de Coquimbo
3,4,Región de Arica y Parinacota
4,5,Región del Maule


In [ ]:
df_dim_region.shape

(16, 2)

In [ ]:
#reemplazamos los valores de la columna sector de la tabla principal por los IDs de los valores unicos
df['RegionUnidad'] = df['RegionUnidad'].replace(df_dim_region.set_index('RegionUnidad')['id_region'])
#Se renombra el titulo de la columna para mejor comprension del cambio
df.rename(columns={'RegionUnidad': 'id_region'}, inplace=True)
#Seteamos a entero el campo
df['id_region'] = df['id_region'].astype(int)
df.shape

(1389235, 97)

In [ ]:
df_dim_region.dtypes

id_region        int64
RegionUnidad    object
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_region.select_dtypes(include=['object']):
    print(columna)
    df_dim_region[columna] = df_dim_region[columna].apply(normalize_text)

RegionUnidad


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_region.to_csv(directorio_tablas_dinamicas + "/dim_region.csv", index=False)

In [ ]:
#Eliminamos las columnas del dataframe principal y se deja solo la columna CodigoExterno
#de esta forma vamos avanzando en la limpieza y reduccion de los datos de la tabla de hecho
df.drop(['Informada','CodigoTipo','TipoConvocatoria','CodigoMoneda','Moneda Adquisicion','EstadoEtapas','TomaRazon',
         'EstadoPublicidadOfertas','JustificacionPublicidad','EstadoCS','Obras','FechaActoAperturaTecnica',
         'FechaActoAperturaEconomica','FechaPublicacion','FechaEstimadaAdjudicacion','FechaSoporteFisico',
         'FechaVisitaTerreno','DireccionVisita','FechaEntregaAntecedentes','DireccionEntrega','FuenteFinanciamiento',
         'ProhibicionContratacion','TipoDuracionContrato','JustificacionMontoEstimado','ObservacionContrato',
         'EsBaseTipo','ValorTiempoRenovacion','PeriodoTiempoRenovacion','EsRenovable','NumeroAprobacion'
         ],axis='columns',inplace=True)
df.shape

(1389235, 68)

#### 9. Tabla **dim_rubros1** *(id_rubro1, Rubro1)*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['Rubro1'].isnull().sum()
cant_nulos

0

In [ ]:
# Obtener valores únicos y eliminar duplicados
unicos_rubros1 = df['Rubro1'].unique()
# Crear un nuevo DataFrame con dos columnas "id_comuna" y "comunaunidad"
df_dim_rubro1 = pd.DataFrame({'id_rubro1': range(1, len(unicos_rubros1) + 1),
                         'Rubro1': unicos_rubros1})

df_dim_rubro1.head(5)

,id_rubro1,Rubro1
0,1,EQUIPAMIENTO PARA LABORATORIOS
1,2,EQUIPAMIENTO Y SUMINISTROS MÉDICOS
2,3,"ARTÍCULOS PARA ESTRUCTURAS, OBRAS Y CONSTRUCCI..."
3,4,VEHÍCULOS Y EQUIPAMIENTO EN GENERAL
4,5,"SERVICIOS BASADOS EN INGENIERÍA, CIENCIAS SOCI..."


In [ ]:
df_dim_rubro1.shape

(58, 2)

In [ ]:
#reemplazamos los valores de la columna sector de la tabla principal por los IDs de los valores unicos
df['Rubro1'] = df['Rubro1'].replace(df_dim_rubro1.set_index('Rubro1')['id_rubro1'])
#Se renombra el titulo de la columna para mejor comprension del cambio
df.rename(columns={'Rubro1': 'id_rubro1'}, inplace=True)
#Seteamos a entero el campo
df['id_rubro1'] = df['id_rubro1'].astype(int)
df.shape

(1389235, 68)

In [ ]:
df_dim_rubro1.dtypes

id_rubro1     int64
Rubro1       object
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_rubro1.select_dtypes(include=['object']):
    print(columna)
    df_dim_rubro1[columna] = df_dim_rubro1[columna].apply(normalize_text)

Rubro1


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_rubro1.to_csv(directorio_tablas_dinamicas + "/dim_rubro1.csv", index=False)

#### 10. Tabla **dim_rubros2** *(id_rubro2, Rubro2)*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['Rubro2'].isnull().sum()
cant_nulos

0

In [ ]:
# Obtener valores únicos y eliminar duplicados
unicos_rubros2 = df['Rubro2'].unique()
# Crear un nuevo DataFrame con dos columnas "id_comuna" y "comunaunidad"
df_dim_rubro2 = pd.DataFrame({'id_rubro2': range(1, len(unicos_rubros2) + 1),
                         'Rubro2': unicos_rubros2})

df_dim_rubro2.head(5)

,id_rubro2,Rubro2
0,1,INSTRUMENTOS DE MEDIDA Y EXPERIMENTACIÓN
1,2,"PRODUCTOS MEDICINALES PARA DEPORTES, PROSTÉTIC..."
2,3,ESTRUCTURAS PREFABRICADAS
3,4,SUMINISTROS Y PRODUCTOS CLÍNICOS
4,5,VEHÍCULOS MOTORIZADOS


In [ ]:
df_dim_rubro2.shape

(342, 2)

In [ ]:
#reemplazamos los valores de la columna sector de la tabla principal por los IDs de los valores unicos
df['Rubro2'] = df['Rubro2'].replace(df_dim_rubro2.set_index('Rubro2')['id_rubro2'])
#Se renombra el titulo de la columna para mejor comprension del cambio
df.rename(columns={'Rubro2': 'id_rubro2'}, inplace=True)
#Seteamos a entero el campo
df['id_rubro2'] = df['id_rubro2'].astype(int)
df.shape

(1389235, 68)

In [ ]:
df_dim_rubro2.dtypes

id_rubro2     int64
Rubro2       object
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_rubro2.select_dtypes(include=['object']):
    print(columna)
    df_dim_rubro2[columna] = df_dim_rubro2[columna].apply(normalize_text)

Rubro2


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_rubro2.to_csv(directorio_tablas_dinamicas + "/dim_rubro2.csv", index=False)

#### 11. Tabla **dim_rubros3** *(id_rubro3, Rubro3)*

In [ ]:
#validamos cantidad de valores nulos en el campo objetivo
cant_nulos = df['Rubro3'].isnull().sum()
cant_nulos

0

In [ ]:
# Obtener valores únicos y eliminar duplicados
unicos_rubros3 = df['Rubro3'].unique()
# Crear un nuevo DataFrame con dos columnas "id_comuna" y "comunaunidad"
df_dim_rubro3 = pd.DataFrame({'id_rubro3': range(1, len(unicos_rubros3) + 1),
                         'Rubro3': unicos_rubros3})

df_dim_rubro3.head(5)

,id_rubro3,Rubro3
0,1,REACTIVOS DE ANALIZADORES CLÍNICOS Y DIAGNÓSTICOS
1,2,PRODUCTOS ORTOPÉDICOS BLANDOS PARA LA EXTREMID...
2,3,"DISPOSITIVOS, ACCESORIOS Y SUMINISTROS PROTÉSICOS"
3,4,CONSTRUCCIONES PREFABRICADAS PARA EMERGENCIAS
4,5,AGUJAS CLÍNICAS


In [ ]:
df_dim_rubro3.shape

(1747, 2)

In [ ]:
#reemplazamos los valores de la columna sector de la tabla principal por los IDs de los valores unicos
df['Rubro3'] = df['Rubro3'].replace(df_dim_rubro3.set_index('Rubro3')['id_rubro3'])
#Se renombra el titulo de la columna para mejor comprension del cambio
df.rename(columns={'Rubro3': 'id_rubro3'}, inplace=True)
#Seteamos a entero el campo
df['id_rubro3'] = df['id_rubro3'].astype(int)
df.shape

(1389235, 68)

In [ ]:
df_dim_rubro3.dtypes

id_rubro3     int64
Rubro3       object
dtype: object

In [ ]:
# Iteramos a través de las columnas y aplicamos la función a las columnas de tipo 'object'
for columna in df_dim_rubro3.select_dtypes(include=['object']):
    print(columna)
    df_dim_rubro3[columna] = df_dim_rubro3[columna].apply(normalize_text)

Rubro3


In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df_dim_rubro3.to_csv(directorio_tablas_dinamicas + "/dim_rubro3.csv", index=False)

-----------------------------------------------------------------------


In [ ]:
df.shape

(1389235, 68)

In [ ]:
# Ahora si podemos contar la cantidad de NaN totales en los registros a eliminar
cols_with_nan = []
nan_for_cols =  df.isnull().sum()
total_nulos = 0
total_registros = len(df)
print('Recuento de valores nulos')
for col in nan_for_cols.index:
  total_nulos += nan_for_cols.get(col)
  porcentaje_nulos = (nan_for_cols.get(col) / total_registros) * 100
  porcentaje_nulos = "{:.2f}%".format(porcentaje_nulos)
  if nan_for_cols.get(col) > 0:
    print(f'{col}: {nan_for_cols.get(col)} = {porcentaje_nulos}%')

print(f'total: {total_nulos}')


Recuento de valores nulos
Contrato: 55689 = 4.01%%
FechaAdjudicacion: 2489 = 0.18%%
UnidadTiempoEvaluacion: 23226 = 1.67%%
Estimacion: 3049 = 0.22%%
MontoEstimado: 1570 = 0.11%%
Tiempo: 1070402 = 77.05%%
ObservacionContrato: 1110027 = 79.90%%
TipoAprobacion: 291199 = 20.96%%
FechaAprobacion: 291199 = 20.96%%
Descripcion linea Adquisicion: 5082 = 0.37%%
NombreProveedor: 75 = 0.01%%
DescripcionProveedor: 4 = 0.00%%
Monto Estimado Adjudicado: 212607 = 15.30%%
FechaEnvioOferta: 10 = 0.00%%
total: 3066628


In [ ]:
#procedemos a eliminar los registros que tengan valore nulos por debajo del 5%
# Eliminar registros donde sector tenga valores vacíos
df = df.dropna(subset=['Contrato','FechaAdjudicacion','UnidadTiempoEvaluacion','Estimacion','MontoEstimado',
                       'Descripcion linea Adquisicion','NombreProveedor','DescripcionProveedor',
                       'FechaEnvioOferta'])

In [ ]:
# Ahora si podemos contar la cantidad de NaN totales en los registros a eliminar
cols_with_nan = []
nan_for_cols =  df.isnull().sum()
total_nulos = 0
total_registros = len(df)
print('Recuento de valores nulos')
for col in nan_for_cols.index:
  total_nulos += nan_for_cols.get(col)
  porcentaje_nulos = (nan_for_cols.get(col) / total_registros) * 100
  porcentaje_nulos = "{:.2f}%".format(porcentaje_nulos)
  if nan_for_cols.get(col) > 0:
    print(f'{col}: {nan_for_cols.get(col)} = {porcentaje_nulos}%')

print(f'total: {total_nulos}')

Recuento de valores nulos
Tiempo: 1001443 = 76.62%%
ObservacionContrato: 1049696 = 80.31%%
TipoAprobacion: 279959 = 21.42%%
FechaAprobacion: 279959 = 21.42%%
Monto Estimado Adjudicado: 206195 = 15.78%%
total: 2817252


In [ ]:
#hacemos las tranformaciones de las columnas restantes si es posible
#Tiempo es entero y de no haber valor se considera que el valor debe ser cero
df['Tiempo'] = df['Tiempo'].fillna(0).astype(int)
df['Tiempo']

<ipython-input-123-1b44f0efe8a7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tiempo'] = df['Tiempo'].fillna(0).astype(int)


0          60
1          60
2          12
3          12
4          12
           ..
1468631     0
1468632     0
1468633     0
1468634     0
1468635     0
Name: Tiempo, Length: 1307068, dtype: int64

In [ ]:
#TipoAprobacion es entero y de no haber valor se considera que el valor debe ser cero
df['TipoAprobacion'] = df['TipoAprobacion'].fillna(0).astype(int)
df['TipoAprobacion']

<ipython-input-124-f6291e742564>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TipoAprobacion'] = df['TipoAprobacion'].fillna(0).astype(int)


0          0
1          0
2          0
3          0
4          0
          ..
1468631    2
1468632    2
1468633    2
1468634    2
1468635    2
Name: TipoAprobacion, Length: 1307068, dtype: int64

In [ ]:
#FechaAprobacion es una fecha y exactamente tiene la misma cantidad de valores nulos
#que tipo de aprobacion 0 por lo que podemos establecer una fecha de 01/01/1900 a los
#que esten tipo 0
df['FechaAprobacion'] = df['FechaAprobacion'].fillna('01-01-1900')
df['FechaAprobacion']

<ipython-input-125-0d451d7373e4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FechaAprobacion'] = df['FechaAprobacion'].fillna('01-01-1900')


0          01-01-1900
1          01-01-1900
2          01-01-1900
3          01-01-1900
4          01-01-1900
              ...    
1468631    2023-04-21
1468632    2023-04-21
1468633    2023-04-21
1468634    2023-04-21
1468635    2023-04-21
Name: FechaAprobacion, Length: 1307068, dtype: object

In [ ]:
df.shape

(1307068, 68)

In [ ]:
#Monto Estimado Adjudicado es entero y esta relacionado a tipo aprobacion por lo que si esta en estado 0
# no tiene monto estimado aun, por lo que se puede establecer a cero
df['Monto Estimado Adjudicado'] = pd.to_numeric(df['Monto Estimado Adjudicado'], errors='coerce')
df = df[~df['Monto Estimado Adjudicado'].isna()]

# Convertir la columna 'Monto Estimado Adjudicado' a tipo entero
df['Monto Estimado Adjudicado'] = df['Monto Estimado Adjudicado'].astype(int)
df['Monto Estimado Adjudicado']

0           61197907
1           61197907
2          117740940
3          117740940
4          117740940
             ...    
1468631     14266151
1468632     14266151
1468633     14266151
1468634     14266151
1468635     14266151
Name: Monto Estimado Adjudicado, Length: 1288670, dtype: int64

In [ ]:
# Ahora si podemos contar la cantidad de NaN totales en los registros a eliminar
cols_with_nan = []
nan_for_cols =  df.isnull().sum()
total_nulos = 0
total_registros = len(df)
print('Recuento de valores nulos')
for col in nan_for_cols.index:
  total_nulos += nan_for_cols.get(col)
  porcentaje_nulos = (nan_for_cols.get(col) / total_registros) * 100
  porcentaje_nulos = "{:.2f}%".format(porcentaje_nulos)
  if nan_for_cols.get(col) > 0:
    print(f'{col}: {nan_for_cols.get(col)} = {porcentaje_nulos}%')

print(f'total: {total_nulos}')

Recuento de valores nulos
ObservacionContrato: 1036863 = 80.46%%
total: 1036863


#### Limpiamos los caracteres especiales y guardamos los archivos de las tablas creadas

In [ ]:
# Guardar la tabla dinamica en la carpeta de tablas
df.to_csv(directorio_tablas_hecho + "/fact_mercado_publico.csv", index=False)